# SETUP

In [ ]:
files   = ['']
sources = ['']

# MODULES

In [56]:
# Standard
import os

# non standard
import pickle
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import astropy.io as ascii
from astropy.io import fits
from scipy.optimize import curve_fit
from importlib import reload

# custom
import nkrpy
from nkrpy import atomiclines
from nkrpy import config
from nkrpy import constants
from nkrpy.functions import *

atomiclines = reload(nkrpy.atomiclines)
listoflines = atomiclines.call(wtype='nir',bu='microns').aperture().return_lines()

In [ ]:
def plotting(ax,xmin,xmax,x,y,count,line,start=False):
    colours = ['orange','black','blue','red',\
              'green','purple','grey']
    colour = colours[count%len(colours)]
    y = np.array(y)
    x = np.array(x)
    origx = x.copy()
    origy = y.copy()
    x = x[~np.isnan(origy)]
    y = y[~np.isnan(origy)]
     
    print("Count: {},Before: {},{}".format(count,x.shape,y.shape))
    if start == False:
        temp = []
        # initially conservative
        if count == 0:
            for i,j in enumerate(x):
                if (j < 1.7):
                    temp.append(i)
        elif count == 1:
            for i,j in enumerate(x):
                if ((1.75 < j) or (j < 1.5)):
                    temp.append(i)
        elif count == 2:
            for i,j in enumerate(x):
                if (1.33 < j) or (j < 1.17):
                    temp.append(i)
        elif count == 3:
            for i,j in enumerate(x):
                if ((j < 1.05) or ((1.11<j)) and (j<1.17)): 
                    temp.append(i)
        elif count == 4:
            for i,j in enumerate(x):
                if (j < 0.95):
                    temp.append(i)
        temp = np.array(temp)
        temp.sort()
        tempx = np.delete(x,temp)
        tempy = np.delete(y,temp)
        expected = [1.,1.]
        params,cov = curve_fit(linear,tempx,tempy,expected)
        if len(temp) > 0:
            y[temp[0]] = linear(x[temp[0]],*params)
            y[temp[len(temp)-1]] = linear(x[temp[len(temp)-1]],*params)
            x = np.delete(x,temp[1:len(temp)-1])
            y = np.delete(y,temp[1:len(temp)-1])

        print("After: {},{}".format(x.shape,y.shape))
        if x.shape[0] == 0:
            x = origx
            y = origy
        count +=1
    ax.plot(x,y,'-',color=colour,label='order {}'.format(count))
    for f in line:
        naming = f
        for pl,pj in enumerate(line[f]):
            if (xmin <= pj <= xmax )and (min(x) <= pj <= max(x) ):
                val = find_nearest(x,pj)[0]
                reg = 15 # region element won't duplicate lines within
                         # +- reg around lines
                if 10<=val<len(x)-11: # line isn't at edge
                    region=y[val-10:val+10] 
                elif val > 10:# right edge
                    region=y[val-10:val]
                elif val<len(x)-11:# left edge
                    region=y[val:val+10]
                else:
                    region = y[val]
                try:
                    linepos = max(region)
                except ValueError:
                    linepos = 5*np.nanmean(x)
                ax.text(pj, linepos*1.05, naming,
                    verticalalignment='bottom',
                    horizontalalignment='center',
                    fontsize=10, color='red',rotation='vertical')

                    ax.plot((pj,pj),(linepos,linepos*1.05),'r-')
    return x,y